In [51]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt

In [52]:
length=400
batch_size=16
learning_rate=0.000001
epocas=5

In [53]:
dados_teste=np.load('dadosTeste.npy')
dadosTreinoValidacao=np.load('dadosTreinoValidacao.npy')

print(np.shape(dadosTreinoValidacao),
np.shape(dados_teste))

dadosTreinoValidacao[5200:5500]

(100000,) (8937,)


array([ 4.,  8.,  1.,  5.,  2.,  1.,  6., 10.,  7.,  4.,  3.,  0.,  4.,
        5.,  3.,  5.,  9.,  9.,  7.,  5.,  4.,  8.,  1.,  4.,  2.,  1.,
        5., 10.,  7.,  4.,  3.,  0.,  4.,  5.,  4.,  5.,  9.,  9.,  7.,
        5.,  4.,  7.,  1.,  4.,  2.,  1.,  5., 10.,  7.,  4.,  3.,  0.,
        4.,  5.,  4.,  5.,  9.,  9.,  7.,  5.,  5.,  7.,  1.,  4.,  2.,
        1.,  6., 10.,  6.,  4.,  3.,  0.,  4.,  5.,  4.,  5.,  9.,  9.,
        7.,  5.,  5.,  7.,  1.,  3.,  2.,  1.,  6., 11.,  6.,  4.,  3.,
        1.,  4.,  5.,  4.,  5.,  9.,  8.,  7.,  5.,  5.,  7.,  1.,  2.,
        2.,  2.,  7., 11.,  6.,  4.,  2.,  1.,  4.,  5.,  4.,  5.,  9.,
        8.,  7.,  5.,  4.,  7.,  1.,  3.,  3.,  2.,  7., 11.,  7.,  4.,
        2.,  2.,  4.,  5.,  4.,  5.,  9.,  7.,  7.,  5.,  4.,  7.,  1.,
        4.,  3.,  2.,  8., 12.,  7.,  4.,  2.,  2.,  4.,  5.,  4.,  5.,
        9.,  7.,  7.,  5.,  5.,  6.,  1.,  3.,  3.,  2.,  8., 12.,  7.,
        3.,  2.,  3.,  4.,  4.,  4.,  5., 10.,  7.,  8.,  6.,  6

In [54]:
percentagem_dados=80

x_train = dadosTreinoValidacao[:80000]

x_validation = dadosTreinoValidacao[80000:]

print(np.shape(x_train), np.shape(x_validation))

(80000,) (20000,)


### Dados treino

In [55]:
from keras.preprocessing.sequence import TimeseriesGenerator

def preparar_targets(x_train, length, batch_size):
    d=5
    salto=3 
    n=d+salto #8
    targets=[]
    for i in range(len(x_train)):
        soma = np.sum(x_train[d:n])
        d= d+1
        n= n+1
        targets.append(soma)
    del targets[-5:]
    
    targets = np.expand_dims(targets, axis=1)
    
    new = x_train[5:]
    inputs = np.expand_dims(new, axis=1)

    time_series_data = tf.keras.preprocessing.sequence.TimeseriesGenerator(inputs, targets, length= length, batch_size=batch_size)
    
    return time_series_data


time_series_data = preparar_targets(x_train,length,batch_size)
#time_series_data[0]

### Dados Validação

In [56]:
validation_data = preparar_targets(x_validation,length,batch_size)
print(len(validation_data[0][0][0]))
validation_data[0][0].shape

400


(16, 400, 1)

## Convolutional Network

In [57]:
length = len(validation_data[0][0][0])
print(length)

400


In [58]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, BatchNormalization


cnnmodel = Sequential([Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(length, 1)),#precisa de dados 3D
                       BatchNormalization(),
                       tf.keras.layers.Activation(tf.keras.activations.relu),
                       MaxPooling1D(pool_size=2),
                       Conv1D(filters=128, kernel_size=2, activation='relu'),
                       BatchNormalization(),       
                       tf.keras.layers.Activation(tf.keras.activations.relu),
                       MaxPooling1D(pool_size=2),
                       Flatten(),
                       Dense(64, activation='relu'),
                       Dense(1)
                      ])
cnnmodel.compile(optimizer= keras.optimizers.Adam(learning_rate=learning_rate), loss='mse', metrics='mae')

#cnnmodel.fit(time_series_data, epochs = epocas,validation_data = validation_data, shuffle= True, verbose=2)

In [59]:
test_targets= preparar_targets(dados_teste,length,batch_size)
test_targets[0][0].shape

(16, 400, 1)

In [60]:
cnnmodel.evaluate(test_targets)

534/534 [==============================] - 2s 4ms/step - loss: 253.2323 - mae: 15.3816


[253.23226928710938, 15.381580352783203]

# Tunning Trial

In [61]:
def build_model(hp):
    cnnmodel = Sequential([Conv1D(filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
                                  kernel_size=2,activation='relu', input_shape=(length, 1)),BatchNormalization()])#precisa de dados 3D
    
    #cnnmodel.add(tf.keras.layers.BatchNormalization())
    cnnmodel.add(tf.keras.layers.Activation(tf.keras.activations.relu))
    
    for i in range(hp.Int('layers_',1,6)):
        cnnmodel.add(MaxPooling1D(pool_size=2))
        cnnmodel.add(Conv1D(filters=hp.Int('conv_1_filter'+str(i), min_value=32, max_value=128, step=16),
                                  kernel_size=hp.Choice('conv_2_kernel', [2,5]),
                                  activation='relu'))

    cnnmodel.add(Flatten())
    cnnmodel.add(Dense(hp.Int('last_dense', min_value=32, max_value=128, step=16), activation='relu'))
    cnnmodel.add(Dense(1))
                 
    cnnmodel.compile(optimizer= keras.optimizers.Adam(learning_rate=learning_rate), loss='mse', metrics='mae')
    
    return cnnmodel

In [62]:
import keras_tuner as kt
tuner2 = kt.Hyperband(
    build_model,
    objective='val_loss',
    max_epochs=5,hyperband_iterations=1,overwrite=True)

In [63]:
tuner2.search(time_series_data,epochs=5,validation_data=validation_data) 

Trial 10 Complete [00h 06m 56s]
val_loss: 24.277807235717773

Best val_loss So Far: 20.007884979248047
Total elapsed time: 03h 46m 11s
INFO:tensorflow:Oracle triggered exit


In [66]:
tuner2.results_summary()

Results summary
Results in .\untitled_project
Showing 10 best trials
Trial summary
Hyperparameters:
conv_1_filter: 96
layers_: 3
conv_1_filter0: 48
conv_2_kernel: 5
last_dense: 112
conv_1_filter1: 128
conv_1_filter2: 96
conv_1_filter3: 48
conv_1_filter4: 64
tuner/epochs: 5
tuner/initial_epoch: 2
tuner/bracket: 1
tuner/round: 1
tuner/trial_id: 0002
Score: 20.007884979248047
Trial summary
Hyperparameters:
conv_1_filter: 96
layers_: 3
conv_1_filter0: 48
conv_2_kernel: 5
last_dense: 112
conv_1_filter1: 128
conv_1_filter2: 96
conv_1_filter3: 48
conv_1_filter4: 64
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 1
tuner/round: 0
Score: 22.097244262695312
Trial summary
Hyperparameters:
conv_1_filter: 32
layers_: 4
conv_1_filter0: 96
conv_2_kernel: 5
last_dense: 80
conv_1_filter1: 64
conv_1_filter2: 128
conv_1_filter3: 32
conv_1_filter4: 64
tuner/epochs: 5
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 24.277807235717773
Trial summary
Hyperparameters:
conv_1_filter: 112
lay